# Full-circle Atomistic Comparison ETS

## Experimental data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!python /home/esrf/polack/ewoks/facets/get_euler.py /data/id01/inhouse/polack/scratch/ewoks_simulation/orientation.txt

In [ ]:
import facets.facets

In [ ]:
from ewoks import execute_graph

In [ ]:
nodes = [
    {
        "id": "fc_analysis",
        "task_type": "class",
        "task_identifier": "facets.facets.FullCircleAnalysis",
    },
    {
        "id": "fc_analysis_sim",
        "task_type": "class",
        "task_identifier": "facets.facets.FullCircleAnalysis",
    }
]

workflow = {"graph": {"id": "hello_world"}, "nodes": nodes}

inputs = [
    {"id": "fc_analysis", "name": "scratch_dirpath", "value": "/data/id01/inhouse/polack/scratch/ewoks_fca/"},
    {"id": "fc_analysis", "name": "vti_filepath", "value": "/data/id01/inhouse/polack/scratch/ewoks_results/S37_structural_properties.vti"},

    {"id": "fc_analysis_sim", "name": "scratch_dirpath", "value": "/data/id01/inhouse/polack/scratch/ewoks_fca/"},
    {"id": "fc_analysis_sim", "name": "xyz_filepath", "value": "/data/id01/inhouse/polack/scratch/ewoks_simulation/exp_minimized.xyz"},
]

Vo

In [ ]:
%matplotlib inline
result = execute_graph(workflow, inputs=inputs)